# asyncio and Concurrent Programming for IO Bound Tasks

foundation for multiple Python asynchronous frameworks:
* high-performance network
* web-servers
* database connection libraries
* distributed task queues

2 levels of APIs available - but we will only explore the high level APIs. The low level APIs are for library and framework development. 

*Warning*, asyncio can get quite complicated when you dive deep.

(High level API)[https://docs.python.org/3.9/library/asyncio-api-index.html]:
* Coroutines and Tasks
* Streams
* Synchronization Primitives
* Subprocesses
* Queues
* Exceptions


## Event loops
Event loop runs in a thread (typically main) and executes all callbacks and tasks

If a task is running, no other task can be running in the same thread

When task executes an await expression: running task is suspended & event loop executes next task.



## coroutines
Code, similar to functions, that are declared with "async def"

"Preffered" way to write asynchio applications

Let's look at Hello World: -run this in a python file or at the python REPL

In [ ]:
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

asyncio.run(main())

Just calling "main()" won't work.  Try it and tell me what happens. 


### 3 way to run a coroutine
* asynchio.run() runs the top-level entry point "main()" function (as shown above)
* Await on a couroutine (as shown below)

In [ ]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

* or thirdly, use asyncio.create_task() function to run coroutines concurrently as asyncio Tasks.

In [3]:
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

"Awaitable" objects are: coroutines, Tasks, Futures and anything else that can be used in an await epression.

coroutines are awaitables and can be awaited from other coroutines as we will see. They come in 2 favors: 

* coroutine function: an async def function;

* coroutine object: an object returned by calling a coroutine function



In [ ]:
import asyncio

async def knights():
    return "Ni!"

async def main():
    # Nothing happens if we just call "knights()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    knights()

    # Let's do it differently now and await it:
    print(await knights())  # will print "Ni!".

asyncio.run(main())

Tasks are created by e.g. asyncio.create_task(). When coroutines are wrapped into a Task, it is scheduled to run... soon. 

Gather collects the tasks from all the awaitables, and returns a list of all the successful tasks. The order is the order of the positional arguments passed to gather. 

Let's look at our fact example but this time concurrently.

In [ ]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
        factorial("D", 5),
    )
    print(L)

asyncio.run(main())

## Exercises:
1. Write a set of tasks to get information from 10-ish web sites asynchronously (import requests):
choose 10 urls

